In [595]:

import pandas as pd
import re

df = pd.read_csv('/Users/yeon/Downloads/hibe_concert_2023.csv')
df.head(3)

,Artist,Date,Place,Price,Seats
0,TXT FANLIVE: PRESENT X TOGETHER,2024.03.02(토) ~ 2024.03.03(일),올림픽공원 SK핸드볼경기장(펜싱경기장),"전석 99,000원",5003석
1,ENHYPEN WORLD TOUR: FATE PLUS [서울],2024.02.24 ~ 2024.02.25,올림픽공원 KSPO DOME(체조경기장),"VIP석 198,000원 M&G석 198,000원 GENERAL석 154,000원",15000석
2,fromis_9 concert: FROM NOW.,2024.01.27(토) ~ 2024.01.28(일),장충체육관,"VIP석 198,000원,154,000원",4507석


## artist컬럼 전처리

In [596]:
df["Artist"] = df["Artist"].str.split(" ",expand=True)[0]
df["Artist"] = df["Artist"].str.replace(',',' ')
df.head(3)

,Artist,Date,Place,Price,Seats
0,TXT,2024.03.02(토) ~ 2024.03.03(일),올림픽공원 SK핸드볼경기장(펜싱경기장),"전석 99,000원",5003석
1,ENHYPEN,2024.02.24 ~ 2024.02.25,올림픽공원 KSPO DOME(체조경기장),"VIP석 198,000원 M&G석 198,000원 GENERAL석 154,000원",15000석
2,fromis_9,2024.01.27(토) ~ 2024.01.28(일),장충체육관,"VIP석 198,000원,154,000원",4507석


## Date컬럼 전처리

### 2024.03.02(토) ~ 2024.03.03(일)형식에서 ~ 기준으로 분리하여 시작일자와 종료일자 컬럼 생성

In [597]:
import re

df= df[df["Date"].str.contains('2023')]
df['start_date'] = df["Date"].str.split("~",expand=True)[0]
df['end_date'] = df["Date"].str.split("~",expand=True)[1]
df['start_date'].replace("토",'', regex=True, inplace=True)
df['start_date']= df['start_date'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
df['end_date'].replace("일",'', regex=True, inplace=True)
df['end_date'] = df['end_date'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)


### start_date 와 end_date의 날짜 형식으로 변환

In [598]:
df['start_date'] = pd.to_datetime(df['start_date'])
df.head(1)

,Artist,Date,Place,Price,Seats,start_date,end_date
3,TOMORROWxTOGETHER,2023.12.02(토) ~ 2023.12.03(일),고척스카이돔,"VIP석 198,000원 일반석 154,000원",16744석,2023-12-02,2023 12 03


In [599]:
df['end_date']= df['end_date'].str.strip() # 공백제거

In [600]:
df['end_date']= pd.to_datetime(df['end_date'])
df['Duration'] = df['end_date'] - df['start_date']
df.head(1)

,Artist,Date,Place,Price,Seats,start_date,end_date,Duration
3,TOMORROWxTOGETHER,2023.12.02(토) ~ 2023.12.03(일),고척스카이돔,"VIP석 198,000원 일반석 154,000원",16744석,2023-12-02,2023-12-03,1 days


### 기간 차이 1일 경우 이틀 공연, 2일 경우 3일 공연임을 나타내기 위해 duration 컬럼 생성

In [601]:
df.loc[df['Duration'] == '1 Days', 'duration'] = '2'
df.loc[df['Duration'] == '2 Days', 'duration'] = '3'

In [602]:
df.drop(['Date','Duration'],axis=1,inplace=True)
df.head(3)

,Artist,Place,Price,Seats,start_date,end_date,duration
3,TOMORROWxTOGETHER,고척스카이돔,"VIP석 198,000원 일반석 154,000원",16744석,2023-12-02,2023-12-03,2
4,LESSERAFIM,잠실종합운동장 실내체육관,"VIP석 198,000원,일반석 154,000원",11032석,2023-08-12,2023-08-13,2
5,SUGA,올림픽공원 KSPO DOME(체조경기장),"VIP석 220,000원165,000",15000석,2023-08-04,2023-08-06,3


## Seats컬럼 전처리

In [603]:
df['Seats']= df['Seats'].str.replace('석','')
df.head(1)

,Artist,Place,Price,Seats,start_date,end_date,duration
3,TOMORROWxTOGETHER,고척스카이돔,"VIP석 198,000원 일반석 154,000원",16744,2023-12-02,2023-12-03,2


## Price컬럼 전처리

### 원을 기준으로 구분하고, VIP와 일반석 가격 컬럼 생성하기

In [605]:
df['vip_price'] = df["Price"].str.split("원",expand=True)[0]
df['standard_Price'] = df["Price"].str.split("원",expand=True)[1]
df['basic_Price'] = df["Price"].str.split("원",expand=True)[2]
df.head(3)

,Artist,Place,Price,Seats,start_date,end_date,duration,vip_price,standard_Price,basic_Price
3,TOMORROWxTOGETHER,고척스카이돔,"VIP석 198,000원 일반석 154,000원",16744,2023-12-02,2023-12-03,2,"VIP석 198,000","일반석 154,000",
4,LESSERAFIM,잠실종합운동장 실내체육관,"VIP석 198,000원,일반석 154,000원",11032,2023-08-12,2023-08-13,2,"VIP석 198,000",",일반석 154,000",
5,SUGA,올림픽공원 KSPO DOME(체조경기장),"VIP석 220,000원165,000",15000,2023-08-04,2023-08-06,3,"VIP석 220,000","165,000",None


In [606]:
df['vip_price'] = df['vip_price'].str.extract(r'(\d+)')
df['standard_Price']= df['standard_Price'].str.extract(r'(\d+)')
df['basic_Price']= df['basic_Price'].str.extract(r'(\d+)')

In [607]:
df_temp = df[['vip_price','standard_Price','basic_Price']]
df_temp = df_temp.astype(float)

In [687]:
df

,Artist,Place,Seats,start_date,end_date,duration,avg_price,sales_expected,Artist_key
3,TOMORROWxTOGETHER,고척스카이돔,16744,2023-12-02,2023-12-03,2,176000,5893888000,0
4,LESSERAFIM,잠실종합운동장 실내체육관,11032,2023-08-12,2023-08-13,2,176000,3883264000,1
5,SUGA,올림픽공원 KSPO DOME(체조경기장),15000,2023-08-04,2023-08-06,3,192000,8640000000,2
6,ENHYPEN,올림픽공원 KSPO DOME(체조경기장),15000,2023-07-29,2023-07-30,2,176000,5280000000,3
7,SEVENTEEN,고척스카이돔,16744,2023-07-21,2023-07-22,2,176000,5893888000,4
8,SUGA,잠실종합운동장 실내체육관,11032,2023-06-24,2023-06-25,2,192000,4236288000,2
9,TOMORROWxTOGETHER,올림픽공원 KSPO DOME(체조경기장),15000,2023-03-25,2023-03-26,2,169000,5070000000,0
10,NewJeans,SK올림픽 핸드볼경기장,5003,2023-07-01,2023-07-02,2,99000,990594000,5
11,LESSERAFIM,올림픽공원 올림픽홀,3032,2023-03-18,2023-03-19,2,99000,600336000,1
12,SEVENTEEN,KSPO DOME,15000,2023-03-10,2023-03-12,3,99000,4455000000,4


In [609]:
df['avg_price'] = df_temp.mean(axis = 'columns')
df['avg_price'] =df['avg_price'].round()
df.drop(['Price','vip_price','standard_Price','basic_Price'],axis=1,inplace=True)

## 위의 결과를 기반으로 예상 매출 컬럼 생성하기

In [610]:
df['Seats'] = df['Seats'].astype(int)
df['duration'] = df['duration'].astype(int)
df['avg_price'] = df['avg_price'].astype(int)
df['avg_price'] = df['avg_price']*1000


In [686]:
df['sales_expected'] = df['Seats'] *  df['avg_price'] *  df['duration']
df.to_csv('/Users/yeon/Downloads/concert.csv')

In [612]:
#데이터프레임.groupby(그룹이되는컬럼)[계산하고 싶은 컬럼].집계함수()
df_sales = df.groupby(df['Artist'])['sales_expected'].sum().reset_index().sort_values(by = 'sales_expected', ascending= False)
df_sales

,Artist,sales_expected
4,SUGA,12876288000
5,TOMORROWxTOGETHER,10963888000
3,SEVENTEEN,10348888000
0,ENHYPEN,5280000000
1,LESSERAFIM,4483600000
2,NewJeans,990594000


## 콘서트 매출 시각화 하기

In [614]:
#월별 소속사 포함 개수 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns


In [613]:
fig = px.pie(data_frame = df_sales, values='sales_expected', names='Artist'
             ,color_discrete_sequence=px.colors.qualitative.Pastel
            ,width=600, height=400
            ,title="[ 하이브 소속 아티스트 2023 콘서트 예상매출 비중 ]"
            ,labels=dict(year_month=" ", title="차트인 횟수"))

fig.update_layout(
                 title_x = 0.5,
                 title_y = 0.9,
                 title_xanchor = "center",
                 title_yanchor = "middle",
                 # 폰트 스타일 추가 부분
                 title_font_size = 16
                , template='plotly_dark')

fig.update_traces(textposition='inside'
                  ,textinfo='percent', textfont_size=10 ,textfont_color="black"
                 )


fig.update_traces(pull=[0.1, 0, 0, 0,0,0])

fig.show()

In [615]:
fig = px.bar(data_frame = df_sales, x= 'Artist', y= 'sales_expected',color='Artist'
            ,color_discrete_sequence=px.colors.qualitative.Pastel
            ,width=600, height=400,text_auto='.2s'
            ,title="[ 하이브 소속 아티스트 2023 콘서트 예상매출액 ]"
            ,labels=dict(year_month=" ", title="차트인 횟수"))

fig.update_layout(
                 title_x = 0.5,
                 title_y = 0.9,
                 title_xanchor = "center",
                 title_yanchor = "middle",
                 # 폰트 스타일 추가 부분
                 title_font_size = 16
                , template='plotly_dark')


fig.update_xaxes(tickangle=45,#x축 각도
                 tickfont_size=10)

fig.update_xaxes(showline=True, linewidth=1, linecolor='white', col=1)
fig.update_yaxes(showline=True, linewidth=1, linecolor='white', col=1)


fig.update_traces(textfont_size=10,textfont_color='black', textangle=0, textposition="inside")
fig.show()



## 멜론 차트와 매핑하기 위한 key값 생성

In [674]:
temp_list = df['Artist'].unique().tolist()
df['Artist_key'] = df['Artist'].map(lambda x : temp_list.index(x))


In [676]:
df.head(3)
df

,Artist,Place,Seats,start_date,end_date,duration,avg_price,sales_expected,Artist_key
3,TOMORROWxTOGETHER,고척스카이돔,16744,2023-12-02,2023-12-03,2,176000,5893888000,0
4,LESSERAFIM,잠실종합운동장 실내체육관,11032,2023-08-12,2023-08-13,2,176000,3883264000,1
5,SUGA,올림픽공원 KSPO DOME(체조경기장),15000,2023-08-04,2023-08-06,3,192000,8640000000,2
6,ENHYPEN,올림픽공원 KSPO DOME(체조경기장),15000,2023-07-29,2023-07-30,2,176000,5280000000,3
7,SEVENTEEN,고척스카이돔,16744,2023-07-21,2023-07-22,2,176000,5893888000,4
8,SUGA,잠실종합운동장 실내체육관,11032,2023-06-24,2023-06-25,2,192000,4236288000,2
9,TOMORROWxTOGETHER,올림픽공원 KSPO DOME(체조경기장),15000,2023-03-25,2023-03-26,2,169000,5070000000,0
10,NewJeans,SK올림픽 핸드볼경기장,5003,2023-07-01,2023-07-02,2,99000,990594000,5
11,LESSERAFIM,올림픽공원 올림픽홀,3032,2023-03-18,2023-03-19,2,99000,600336000,1
12,SEVENTEEN,KSPO DOME,15000,2023-03-10,2023-03-12,3,99000,4455000000,4


In [662]:
df_merge = pd.merge(left = df_melon , right = df_sales , how = "left", on = "Artist_key").sort_values(by = ['title','sales_expected'] , ascending = False)
df_merge.drop(['Artist'], axis = 1)
df_merge.to_csv('/Users/yeon/Downloads/concert_pre.csv')

,artist,Artist_key,title,sales_expected
3,방탄소년단,2,120,1.287629e+10
1,NewJeans,5,110,9.905940e+08
0,LE SSERAFIM (르세라핌),1,54,4.483600e+09
4,세븐틴 (SEVENTEEN),4,26,1.034889e+10
5,투모로우바이투게더,0,8,1.096389e+10
6,프로미스나인,7,2,NaN
2,TWS (투어스),8,1,NaN


In [656]:
pd.merge(left = df_melon , right = df_sales , how = "left", on = "Artist_key").sort_values(by = ['sales_expected','title'] , ascending = False)

,artist,Artist_key,title,Artist,sales_expected
3,방탄소년단,2,120,SUGA,1.287629e+10
5,투모로우바이투게더,0,8,TOMORROWxTOGETHER,1.096389e+10
4,세븐틴 (SEVENTEEN),4,26,SEVENTEEN,1.034889e+10
0,LE SSERAFIM (르세라핌),1,54,LESSERAFIM,4.483600e+09
1,NewJeans,5,110,NewJeans,9.905940e+08
6,프로미스나인,7,2,NaN,NaN
2,TWS (투어스),8,1,NaN,NaN
